In [ ]:
import pandas as pd
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics
import os
import SimpleITK as sitk
import nibabel as nib
from radiomics import getTestCase
import porespy as ps
import pandas as pd
import numpy as np
from skimage import morphology
from radiomics import shape
import pickle
from radiomics import shape, firstorder
import plotly.express as px

In [ ]:
sys.path.insert(1, "/Users/dvarelat/Documents/MASTER/TFM/methods")
import cardiac_region
import importlib
importlib.reload(cardiac_region)
import cardiac_region as c

In [ ]:
ESPECIMEN = "20190517_E1"

In [ ]:
cellpose_nu = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/nuclei/{ESPECIMEN}_MASK_EQ_XYZ_decon.nii.gz"
gasp_mem = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/membranes/GASP_PNAS/{ESPECIMEN}_mGFP_XYZ_predictions_GASP.nii.gz"
pickle_div = f"/Users/dvarelat/Documents/MASTER/TFM/methods/division/LISTS_NUCLEI/{ESPECIMEN}_list_nuclei.pkl"
nuclei = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/DECON_05/DAPI/{ESPECIMEN}_DAPI_decon_0.5.nii.gz"

In [ ]:
DAPI = nib.load(nuclei).get_fdata()
DAPI = DAPI[:,:,:,0]

In [ ]:
pred_nu = nib.load(cellpose_nu).get_fdata()
pred_mem = nib.load(gasp_mem).get_fdata()

In [ ]:
mask_mem = np.where(pred_mem != 0, True, False)
mask_on_nuclei = mask_mem * pred_nu

In [ ]:
props_nu = ps.metrics.regionprops_3D(morphology.label(mask_on_nuclei))
len(props_nu)

In [ ]:
with open(pickle_div, "rb") as f:
    list_div = pickle.load(f)
len(list_div)

In [ ]:
centroids_nu = [[round(i) for i in p["centroid"]] for p in props_nu]
NU_original_labels_centroids = [pred_nu[c[0], c[1], c[2]] for c in centroids_nu]

In [ ]:
df = pd.DataFrame(
            {
                "cell_in_props": range(len(props_nu)),
                "volumes": [p.volume for p in props_nu],
                "original_labels": NU_original_labels_centroids,
                "centroids": centroids_nu,
            }
        )
df.describe()
print(np.median(df.volumes))
np.mean(df.volumes)

In [ ]:
df = df[df.original_labels != 0]
df.describe()

In [ ]:
print(np.median(df.volumes))
np.mean(df.volumes)

In [ ]:
med = np.median(df.volumes)
print(0.2 * med)
print(10 * med)

In [ ]:
df = df[df.volumes > 0.2 * med]
df = df[df.volumes < 10 * med]

In [ ]:
df.describe()

In [ ]:
dfnew = pd.DataFrame(results)
df_ = df.merge(dfnew, left_on='cell_in_props', right_on='cell')

In [ ]:
div = [1 if i in list_div else 0 for i in list(df_.cell_in_props) ]

In [ ]:
df_["div"] = div

In [ ]:
df_.groupby(["div"]).count()

In [ ]:
df_

In [ ]:
cols = ['Elongation', 'Flatness', 'LeastAxisLength', 'MajorAxisLength',
       'Maximum2DDiameterColumn', 'Maximum2DDiameterRow',
       'Maximum2DDiameterSlice', 'Maximum3DDiameter', 'MeshVolume',
       'MinorAxisLength', 'Sphericity', 'SurfaceVolumeRatio',
       'VoxelVolume', '10Percentile', '90Percentile', 
       'Entropy', 'InterquartileRange', 'Kurtosis', 'Maximum',
       'MeanAbsoluteDeviation', 'Mean', 'Median', 'Minimum', 'Range',
       'RobustMeanAbsoluteDeviation', 'RootMeanSquared', 'Skewness', 'Uniformity']

In [ ]:
melted = pd.melt(df_, id_vars=["cell", "div"], 
        value_vars=cols)

In [ ]:
fig = px.box(melted , x="variable", y="value", color="div")
fig.show()

In [ ]:
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
for c in cols:
    print(c)
    model = ols('value ~ C(div)', data=melted[melted.variable == c]).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(anova_table["PR(>F)"]["C(div)"])
    print("---------")

## ALL

In [ ]:
list_dfs=[]
especimens = [
    "20190504_E1",
    "20190404_E2",
    "20190520_E4",
    "20190516_E3",
    "20190806_E3",
    "20190520_E2",
    "20190401_E3",
    "20190517_E1",
    "20190520_E1",
    "20190401_E1",
]

order = [1,7,10,3,4,2,5,8,9,6]

for i, e in enumerate(especimens):
    print(e)
    f = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/NUCLEI/{e}_NUCLEI_properties.csv"
    df = pd.read_csv(f)
    print(df.shape)
    df["especimen"] = e
    #df["stage"] = str(dict_order[i])
    #df["stageNUMBER"] = dict_order[i]
    list_dfs.append(df)

In [ ]:
dfall = pd.concat(list_dfs)
print(dfall.lines.unique())

In [ ]:
dict_lines = dict(zip(dfall.lines.unique(), ["back", "paraxial", "splanchnic", "somatic", "myocardium","myocardium" , "myocardium" ]) ) 
dfall["name_lines"] = dfall["lines"].apply(lambda x: dict_lines[x])

In [ ]:
dfall.groupby(["div"]).count()

In [ ]:
d = {}
for i,e in enumerate(especimens):
    print(e)
    df = dfall[dfall.especimen == e]
    print(df.shape)
    d[e] = df.groupby(["name_lines"]).count()["volumes"].to_dict()
    d[e]["stageNUMBER"] = dict_order[i]
    d[e]["stage"] = str(dict_order[i])
    d[e]["totalNum"] = np.sum(list(df.groupby(["name_lines"]).count()["volumes"].to_dict().values()))
df_numbers = pd.DataFrame(d).T
df_numbers ["proportion myo/spl"] = df_numbers.apply(lambda x: x["myocardium"]/x["splanchnic"], axis=1)
df_numbers ["proportion myo/total"] = df_numbers.apply(lambda x: x["myocardium"]/x["totalNum"], axis=1)
dforder_numbers = df_numbers.sort_values(by=['stageNUMBER'])

In [ ]:
melted = pd.melt(dforder_numbers, id_vars=["stage"], 
        value_vars=["back", "paraxial", "somatic", "splanchnic", "myocardium", "myocardium", "totalNum"])
melted.head(3)

In [ ]:
fig = px.line(melted, x="stage", y="value", color="variable", markers=True)
fig.show()

In [ ]:
fig = px.scatter(dforder_numbers, x="stage", y="proportion myo/total", trendline="ols")
fig.show()

In [ ]:
dforder = dfall.sort_values(by=['stageNUMBER'])

In [ ]:
shape = [ 'Elongation', 'Flatness', 'LeastAxisLength', 'MajorAxisLength',
       'Maximum2DDiameterColumn', 'Maximum2DDiameterRow',
       'Maximum2DDiameterSlice', 'Maximum3DDiameter', 'MeshVolume',
       'MinorAxisLength', 'Sphericity', 'SurfaceArea', 'SurfaceVolumeRatio',
       'VoxelVolume']
for var in shape:
    px.box(dforder, x="stage", y=var, color="name_lines").show()

In [ ]:
dfall.columns

In [ ]:
intensity = [ '10Percentile', '90Percentile', 'Energy',
       'Entropy', 'InterquartileRange', 'Kurtosis', 'Maximum',
       'MeanAbsoluteDeviation', 'Mean', 'Median', 'Minimum', 'Range',
       'RobustMeanAbsoluteDeviation', 'RootMeanSquared', 'Skewness',
       'TotalEnergy', 'Uniformity', 'Variance',]
for var in intensity:
    px.box(dforder, x="stage", y=var, color="name_lines").show()

In [ ]:
colsDIV = ['Elongation', 'Flatness', 'LeastAxisLength', 'MajorAxisLength',
       'Maximum2DDiameterColumn', 'Maximum2DDiameterRow',
       'Maximum2DDiameterSlice', 'Maximum3DDiameter', 
       'MinorAxisLength', 'Sphericity', 'SurfaceVolumeRatio', '10Percentile', '90Percentile', 
       'Entropy', 'InterquartileRange', 'Kurtosis', 'Maximum',
       'MeanAbsoluteDeviation', 'Mean', 'Median', 'Minimum', 'Range',
       'RobustMeanAbsoluteDeviation', 'RootMeanSquared', 'Skewness', 'Uniformity']
division = pd.melt(dfall, id_vars=["cell", "div"], 
        value_vars=colsDIV)
fig = px.box(division , x="variable", y="value", color="div")
fig.show()

In [ ]:
73952-73795